<a href="https://colab.research.google.com/github/SeaJungg/bigdata_analytics2021/blob/master/p%EB%B0%B8%EB%A5%98%EC%97%90%EB%AF%B8%EC%B3%90%EC%84%9C_%EB%8F%8C%EC%95%84%EC%98%A8_%EC%84%B8%EC%95%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files 

In [ ]:
from google.colab import files 
upload_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/secom.data', sep = ' ', header = None )
upload_label = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/secom_labels.data', sep = ' ', header = None )

In [ ]:
import io

In [ ]:
df = pd.DataFrame(upload_data)
label = upload_label[0]

In [ ]:
col_candidate = []
#COl 기준 std가 0이면 제외하기
for idx, val in zip(df.columns, df.describe().iloc[2]):
  if val != 0:
    col_candidate.append(idx)
df = df[col_candidate]
print(len(col_candidate))

#COl 기준 데이터의 수가 55% 이상인 것만 살리기
col_candidate = []
for idx, val in zip(df.columns, df.describe().iloc[0]):
  if val >= 700:
    col_candidate.append(idx)
df = df[col_candidate]
print(len(col_candidate))

474
450


In [ ]:
!pip install impyute

import copy
data_with_label = copy.deepcopy(df)
data_with_label['Class'] = label

from sklearn.impute import KNNImputer

#결측치 채우기 with KNNImputer
imputer = KNNImputer(n_neighbors=3)
imputed = imputer.fit_transform(data_with_label)
data_with_label_imputed = pd.DataFrame(imputed, columns=data_with_label.columns)

In [ ]:
#nan 남아있는지 확인
check_for_any_nan= data_with_label_imputed .isna().values.any()
print("NaN Presence:"+str(check_for_any_nan))

In [ ]:
from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
print(data_with_label_imputed.columns)
#걍 시험삼아 첫번째 열에다가 회귀분석 실시.
test_col = data_with_label_imputed[0]
test_col = pd.concat([test_col,label],axis=1)
test_col.columns = ['test_col','Class']
check_for_any_nan= test_col .isna().values.any()
print("NaN Presence:"+str(check_for_any_nan))
temp =[]
res = ols('Class~test_col', data=test_col).fit()
print(res.summary())

Index([      0,       1,       2,       3,       4,       6,       7,       8,
             9,      10,
       ...
           577,     582,     583,     584,     585,     586,     587,     588,
           589, 'Class'],
      dtype='object', length=451)
NaN Presence:False
                            OLS Regression Results                            
Dep. Variable:                  Class   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9425
Date:                Tue, 01 Jun 2021   Prob (F-statistic):              0.332
Time:                        16:26:13   Log-Likelihood:                -1130.1
No. Observations:                1567   AIC:                             2264.
Df Residuals:                    1565   BIC:                             2275.
Df Model:                           1                                         
Covariance Type:

In [ ]:
res.tvalues

Intercept   -0.710872
test_col    -0.970829
dtype: float64

In [ ]:
res.pvalues[1]

0.0022796026874769572

In [ ]:
print(res.fvalue) #F-statistic
print(res.f_pvalue) #Prob (F-statistic)

0.9425082852447177
0.33178362551515383


In [ ]:
tvalue = []
fstatic = []
f_pvalue = []
pvalue = []

In [ ]:
for i in data_with_label_imputed.columns.values:
  test_col = data_with_label_imputed[i]
  test_col = pd.concat([test_col,label],axis=1)
  test_col.columns = ['test_col','Class']
  res = ols('Class~test_col', data=test_col).fit()
  pvalue.append([i,res.pvalues[1]])
  tvalue.append([i,res.tvalues])
  fstatic.append([i,res.fvalue])
  f_pvalue.append([i,res.f_pvalue])

# P value <= 0.001인 Feature만 선택 
*** 선정된 Feature : 21 	28  59  103  121  124  125  129  160  163  164  165  210  295  298  299  300  316  348  430  431  434  435  436  510 **

In [ ]:
col_names = []
for i in pvalue:
  if i[1] <= 0.001 :
    col_names.append(i[0])
print(len(col_names))

26


In [ ]:
data_pval = data_with_label_imputed[col_names]
data_pval.head()

,21,28,59,103,121,124,125,129,160,163,164,165,210,295,298,299,300,316,348,430,431,434,435,436,510,Class
0,-5419.00,64.2333,-1.7264,-0.0042,15.88,15.93,0.8656,-0.0473,967.0,0.090,0.048,0.095,0.0772,398.3185,0.0373,0.0202,0.0462,6.2698,0.0215,18.7673,33.1562,10.0503,2.7073,3.1158,64.6707,-1.0
1,-5441.50,68.4222,0.8073,-0.0045,15.88,15.88,0.8703,-0.0946,59.0,0.112,0.115,0.124,0.0566,26.5879,0.0502,0.0561,0.0591,5.6522,0.0142,10.4383,2.2655,8.6336,5.7093,1.6779,141.4365,-1.0
2,-5447.75,67.1333,23.8245,-0.0026,15.90,15.95,0.8798,-0.1892,788.0,0.187,0.117,0.068,0.0339,329.6406,0.0800,0.0583,0.0326,5.7247,0.0105,10.3162,29.1663,14.2503,5.7650,0.8972,240.7767,1.0
3,-5468.25,62.9333,24.3791,-0.0059,15.55,15.59,1.3660,0.2838,355.0,0.068,0.108,0.100,0.1248,157.0889,0.0285,0.0445,0.0465,5.4440,0.0407,15.7089,13.4051,5.1760,5.3899,1.3671,113.5593,-1.0
4,-5476.25,62.8333,-12.2945,-0.0045,15.75,15.71,0.9460,-0.5677,283.0,0.147,0.040,0.113,0.0915,128.0296,0.0755,0.0181,0.0476,4.8956,0.0246,12.7642,10.7390,11.4064,2.0088,1.5533,148.0663,-1.0


In [ ]:
!pip install imblearn

In [ ]:
from sklearn.model_selection import train_test_split
X = data_pval.drop(['Class'], axis=1)
y = data_pval['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state = 1)

In [ ]:
pip install sklearn

In [ ]:
#RobustScaler : - 중앙값 = 0 / IQR(1분위(25%) ~ 3분위(75%)) = 1
#   - 이상치(outlier) 영향 최소화
#   - 더 넓게 분포
# 중앙값 0 / IQR 1
from sklearn.preprocessing import RobustScaler
transformer = RobustScaler()
X_train  = transformer.fit_transform(X_train)
X_test = transformer.fit_transform(X_test)



#   - 표준화 Standardization
#StandardScaler: - 평균 = 0 / 표준편차 = 1
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler()   
#scaler = scaler.fit_transform(data)
# 교차검증시
#scaler.fit(X_train)
#X_train = scaler.transform(X_train)
#X_test = scaler.transform(X_test)


#MinMaxScaler : - 최대값 = 1 / 최소값 = 0
#   - 최소-최대 정규화 Min-Max Normalization
#   - 이상치에 취약하다.
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#data_pval_minmax = pd.DataFrame(data = scaler.fit_transform(data_pval.values), columns = data_pval.columns)

SyntaxError: ignored

In [ ]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=0).fit_resample(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score # 정확도 함수
from sklearn.model_selection import cross_val_predict

clf = RandomForestClassifier(max_depth = 4, min_samples_leaf = 4, min_samples_split = 4, n_estimators = 80, random_state = 0)
clf.fit(X_resampled, y_resampled)

predict_RF = clf.predict(X_test)
print(accuracy_score(y_test,predict_RF))

0.7664543524416136


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
confusion_matrix(y_test, predict_RF)

array([[339, 101],
       [  9,  22]])

In [ ]:
accuracy_score(y_test,predict_RF)

0.7664543524416136

In [ ]:
precision_score(y_test,predict_RF)

0.17886178861788618

In [ ]:
recall_score(y_test,predict_RF)

0.7096774193548387

In [ ]:
f1_score(y_test,predict_RF)

0.28571428571428575